In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('seinet_results.csv', index_col=0)
df.drop(columns = ['Unnamed: 0', 'seinet_status'], inplace = True)
print(df.shape)
print(df.columns)

(1057, 5)
Index(['Genus', 'Species', 'FNA', 'VPAP', 'SW Field Guide'], dtype='object')


# Lifecycle duration

In [3]:
def extract_life_duration(text):
    """
    Extract plant life duration from descriptive text.

    Parameters
    ----------
    text : str or None

    Returns
    -------
    str
        One of:
        'annual', 'biennial', 'perennial',
        'annual/biennial', 'annual/perennial',
        'biennial/perennial', 'unknown'
    """
    if not text or not isinstance(text, str):
        return "unknown"

    t = text.lower()

    found = set()

    # strict word boundaries to avoid false matches
    patterns = {
        "annual": r"\bannual\b",
        "biennial": r"\bbiennial\b",
        "perennial": r"\bperennial\b"
    }

    for label, pattern in patterns.items():
        if re.search(pattern, t):
            found.add(label)

    if not found:
        return "unknown"

    # normalize combinations
    if found == {"annual"}:
        return "annual"
    if found == {"biennial"}:
        return "annual"
    if found == {"perennial"}:
        return "perennial"
    if found == {"annual", "biennial"}:
        return "annual"
    if found == {"annual", "perennial"}:
        return "annual/perennial"
    if found == {"biennial", "perennial"}:
        return "perennial"

    # rare but possible
    return "-".join(sorted(found))

In [4]:
df['duration_FNA'] = df["FNA"].apply(extract_life_duration)
df['duration_VPAP'] = df["VPAP"].apply(extract_life_duration)
df['duration_SWFG'] = df["SW Field Guide"].apply(extract_life_duration)

In [5]:
def consensus_life_duration(row):
    values = [
        row["duration_FNA"],
        row["duration_VPAP"],
        row["duration_SWFG"]
    ]
    values = [v for v in values if v != "unknown"]

    if not values:
        return "unknown"

    # if all agree
    if len(set(values)) == 1:
        return values[0]

    # otherwise keep ambiguity explicit
    return "/".join(sorted(set(values)))

In [6]:
df["duration_consensus"] = df.apply(consensus_life_duration, axis=1)

In [7]:
df.loc[df["duration_consensus"] == "annual/annual-biennial-perennial", "duration_consensus"] = "annual"
df.loc[df["duration_consensus"] == "annual-biennial-perennial/perennial", "duration_consensus"] = "perennial"
df.loc[df["duration_consensus"] == "annual-biennial-perennial", "duration_consensus"] = "annual"
df.loc[(df['Genus'] == 'Baileya') & (df['Species'] == 'multiradiata'), 'duration_consensus'] = "perennial"
df.loc[df["duration_consensus"] == "annual/perennial/perennial", "duration_consensus"] = "perennial"
df.loc[df["duration_consensus"] == "annual/annual/perennial", "duration_consensus"] = "annual"

In [8]:
df.loc[(df['Genus'] == 'Atriplex') & (df['Species'] == 'elegans'), 'duration_consensus'] = "annual"
df.loc[(df['Genus'] == 'Ambrosia') & (df['Species'] == 'confertiflora'), 'duration_consensus'] = "perennial"
df.loc[(df['Genus'] == 'Sonchus') & (df['Species'] == 'oleraceus'), 'duration_consensus'] = "annual"
df.loc[(df['Genus'] == 'Xanthisma') & (df['Species'] == 'spinulosum'), 'duration_consensus'] = "perennial"
df.loc[(df['Genus'] == 'Cryptantha') & (df['Species'] == 'holoptera'), 'duration_consensus'] = "perennial"
df.loc[(df['Genus'] == 'Cuscuta') & (df['Species'] == 'umbellata'), 'duration_consensus'] = "annual"
df.loc[(df['Genus'] == 'Chamaesyce') & (df['Species'] == 'pediculifera'), 'duration_consensus'] = "perennial"
df.loc[(df['Genus'] == 'Ditaxis') & (df['Species'] == 'neomexicana'), 'duration_consensus'] = "perennial"

In [9]:
lifespan = ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
            'annual', 'perennial', 'unknown', 'annual', 'perennial', 'annual', 'unknown', 'unknown', 'perennial', 'unknown', 
            'unknown', 'unknown', 'annual', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'perennial', 
            'unknown', 'unknown', 'unknown', 'unknown', 'annual', 'unknown', 'annual', 'annual', 'unknown', 'unknown', 
            'unknown', 'annual', 'annual', 'unknown', 'unknown', 'unknown', 'unknown', 'perennial', 'unknown', 
            'unknown', 'unknown', 'unknown', 'annual', 'annual', 'unknown']

In [10]:
for i, j in zip(df[df["duration_consensus"] == 'annual/perennial'][['Genus', 'Species']].values, lifespan):
    df.loc[(df['Genus'] == i[0]) & (df['Species'] == i[1]), 'duration_consensus'] = j

In [11]:
lifespan2 = ['perennial', 'annual', 'perennial', 'annual', 'perennial', 'perennial', 'perennial', 'perennial', 'perennial', 'annual', 
             'perennial', 'perennial', 'annual', 'perennial', 'annual', 'perennial', 'annual', 'perennial', 'perennial', 'annual', 
             'perennial', 'annual', 'perennial', 'annual', 'perennial', 'annual', 'perennial', 'annual', 'perennial', 'perennial']

In [12]:
for i, j in zip(df[(df.duration_consensus == 'unknown') & (df.duration_SWFG == 'annual/perennial')][['Genus', 'Species']].values, lifespan2):
    df.loc[(df['Genus'] == i[0]) & (df['Species'] == i[1]), 'duration_consensus'] = j

In [13]:
lifespan3 = ['perennial', 'unknown',  'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'perennial', 'perennial', 'unknown', 'unknown', 'unknown', 'perennial', 'annual', 'perennial', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'perennial', 'perennial', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'perennial', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'perennial',  'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'annual', 'unknown', 'annual', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'perennial', 'unknown', 'perennial', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown']

In [14]:
for i, j in zip(df[df.duration_consensus == 'unknown'][['Genus', 'Species']].values, lifespan3):
    df.loc[(df['Genus'] == i[0]) & (df['Species'] == i[1]), 'duration_consensus'] = j

In [15]:
lifespan4 = ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'annual', 'unknown', 'unknown', 
             'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 
             'perennial', 'perennial', 'perennial', 'perennial', 'unknown', 'unknown', 'unknown', 'perennial', 'perennial', 'annual', 
             'unknown', 'unknown', 'unknown', 'unknown', 'perennial', 'perennial', 'perennial', 'perennial', 'perennial', 'perennial', 
             'unknown', 'annual', 'unknown', 'unknown', 'unknown', 'perennial', 'perennial', 'perennial', 'perennial', 'perennial', 
             'perennial', 'annual', 'unknown', 'annual', 'annual', 'annual', 'unknown', 'annual', 'perennial', 'perennial', 
             'unknown', 'unknown', 'unknown', 'annual', 'annual', 'annual', 'annual', 'perennial', 'unknown', 'unknown', 
             'unknown', 'unknown', 'perennial', 'perennial', 'perennial', 'perennial', 'perennial', 'perennial', 'perennial', 'annual', 
             'unknown', 'perennial']

In [16]:
for i, j in zip(df[df.duration_consensus == 'unknown'][['Genus', 'Species']].values, lifespan4):
    df.loc[(df['Genus'] == i[0]) & (df['Species'] == i[1]), 'duration_consensus'] = j

In [30]:
lifespan5 = ['perennial', 'annual', 'perennial', 'annual', 'annual', 'annual', 'perennial', 'annual', 'perennial', 'annual', 
             'unknown', 'perennial', 'annual', 'annual', 'annual', 'annual', 'perennial', 'annual', 'annual', 'perennial', 
             'perennial', 'annual', 'unknown', 'annual', 'annual', 'annual', 'perennial', 'annual', 'perennial', 'annual', 
             'perennial', 'perennial', 'perennial', 'annual', 'unknown', 'perennial', 'perennial', 'perennial', 'annual', 'annual']

In [32]:
for i, j in zip(df[df.duration_consensus == 'unknown'][['Genus', 'Species']].values, lifespan5):
    df.loc[(df['Genus'] == i[0]) & (df['Species'] == i[1]), 'duration_consensus'] = j

In [36]:
df = df[df.duration_consensus != 'unknown'].reset_index(drop=True)

In [37]:
df.duration_consensus.value_counts()

duration_consensus
perennial    579
annual       475
Name: count, dtype: int64

In [39]:
df.drop(columns = ['duration_FNA', 'duration_VPAP', 'duration_SWFG'], inplace = True)

# Native

In [56]:
NON_NATIVE_TERMS = [
    "non-native",
    "introduced",
    "invasive",
    "exotic",
    "naturalized"
]

def parse_nativity(sw_text):
    """
    Determine nativity from SW Field Guide text.

    Parameters
    ----------
    sw_text : str or None

    Returns
    -------
    str
        'native', 'non-native', or 'unknown'
    """

    if pd.isna(sw_text):
        return "unknown"

    text = sw_text.lower()

    # --- Explicit Nativity field ---
    nativity_match = re.search(
        r"nativity\s*:\s*([a-z\-]+)",
        text
    )
    
    if nativity_match:
        value = nativity_match.group(1)

        if value == "native":
            return "native"

        if value in NON_NATIVE_TERMS:
            return "non-native"

    # --- Fallback: search for non-native indicators anywhere ---
    for term in NON_NATIVE_TERMS:
        if re.search(rf"\b{term}\b", text):
            return "non-native"

    return "unknown"

In [57]:
df["nativity_sw"] = df["SW Field Guide"].apply(parse_nativity)

In [58]:
df["nativity_sw"].value_counts()

nativity_sw
native        788
unknown       156
non-native    110
Name: count, dtype: int64

In [60]:
df["VPAP"].apply(parse_nativity).value_counts()

VPAP
unknown       1049
non-native       5
Name: count, dtype: int64

In [61]:
df["FNA"].apply(parse_nativity).value_counts()

FNA
unknown       964
non-native     90
Name: count, dtype: int64

# Elevation

# Aquatic

# Lifeform